# STEP3 TA IndéMode  データ取得 周辺人口

>### 環境構築

ターミナルにて  
 ● pip install requests  
 ● pip install urllib3  
 ● pip install beautifulsoup4  
 ● pip install gspread  
 ● pip install google-auth

>### 活用API

●　国土地理院API : 住所を緯度経度に変換  
●　jSTAT MAP API ：商圏分析

>### コード

In [3]:
# ライブラリインポート
import requests
import urllib.request
import pandas as pd
import gspread 
from google.oauth2 import service_account
import json
import sqlite3

# 国土地理院APIから緯度経度を取得
Address = "東京都千代田区有楽町2-2-13"
makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
s_quote = urllib.parse.quote(Address)
response = requests.get(makeUrl + s_quote)

#緯度を表示
#print(response.json()[0]["geometry"]["coordinates"][0])
#経度を表示
#print(response.json()[0]["geometry"]["coordinates"][1])

#jSTAT MAP認証設定
REQUEST_URL = 'https://jstatmap.e-stat.go.jp/statmap/api/1.00?category=richReport&func=getSummary'
USER_ID = '&userid=noriyasukawana@outlook.jp'  #個人の登録ID
API_KEY = '&key=dMUbbbyc9ThTzG4PNpA2'  #個人のAPIキー

# params入力設定
latitude = '&lat=' + str(response.json()[0]["geometry"]["coordinates"][1])  # 緯度
longitude = '&lng=' + str(response.json()[0]["geometry"]["coordinates"][0])  # 経度
rangeType = '&rangeType=driveTime'  # circle(円) or driveTime(到達圏)
travelMode = '&travelMode=walking'  # car(車) or walking(徒歩)
speed = '&speed=3.2'  # 時速(km/h)
time = '&time=15,30,45'  # 移動時間(min)
output = '&output=json'  # 出力形式
radius = '&radius=500'

# リクエスト
res = requests.get(REQUEST_URL + USER_ID + latitude + longitude + rangeType + travelMode + speed + time + API_KEY + output)
result = res.json()

#性別別人口
Gender_pop = result['GET_SUMMARY']['DATASET_INF'][0]['TABLE_INF'][0]['DATA_INF']['VALUE']
Gender_pop2 = [{key: value for key, value in entry.items() if key not in ['@cat11', '@cat12']} for entry in Gender_pop]
Gender_pop3 = [entry['$'] for entry in Gender_pop2]
# リストを3つずつに分割
list_Gender = [Gender_pop3[i:i+3] for i in range(0, len(Gender_pop3), 3)]
# DataFrameに変換
df_Gender = pd.DataFrame(list_Gender, columns=['総人口', '男性人口', '女性人口'])
#display(df_Gender)

#年齢別人口
Age_pop = result['GET_SUMMARY']['DATASET_INF'][0]['TABLE_INF'][1]['DATA_INF']['VALUE']
Age_pop2 = [{key: value for key, value in entry.items() if key not in ['@cat11', '@cat13']} for entry in Age_pop]
Age_pop3 = [entry['$'] for entry in Age_pop2]
# リストを3つずつに分割
list_Age = [Age_pop3[i:i+16] for i in range(0, len(Age_pop3), 16)]
# DataFrameに変換
df_Age = pd.DataFrame(list_Age, columns=['4 歳以下', '5～9 歳', '10～14 歳', '15～19 歳', '20～24 歳', '25～29 歳', '30～34 歳','35～39 歳', '40～44 歳','45～49 歳', '50～54 歳','55～59 歳', '60～64 歳','65～69 歳', '70～74 歳','75 歳以上'])
#display(df_Age)

# DataFrameを結合
df = pd.concat([df_Gender, df_Age], axis=1)
#display(df)

# 項目作成
class_name = result['GET_SUMMARY']['DATASET_INF'][0]['TABLE_INF'][0]['CLASS_INF']['CLASS_OBJ'][0]['CLASS']
class_name2 = [{key: value for key, value in entry.items() if key not in ['@code']} for entry in class_name]
class_name3 = [entry['@name'] for entry in class_name2]

# 項目カラムを用意
item_column = ['徒歩15分圏内', '徒歩30分圏内', '徒歩45分圏内', class_name3[3], class_name3[4]]

# 項目カラムをDataFrameの先頭列に追加
df.insert(0, '項目', item_column)
#display(df)


# スプレッドシートにデータ保存
# スプレッドシートの認証設定
SP_CREDENTIAL_FILE = 'gspread-test-399313-ca3a69c4d75e.json' #個人の認証キーのjsonファイルのパスを記入
SP_COPE = [
    'https://www.googleapis.com/auth/drive',
    'https://spreadsheets.google.com/feeds'
]
credentials = service_account.Credentials.from_service_account_file(SP_CREDENTIAL_FILE, scopes=SP_COPE)
gc = gspread.authorize(credentials)

#スプレッドシート/シート指定
SP_SHEET_KEY = '1474-fT6RkuGOvPwjyCJIEtD6qrNjI6EP2bw3vtHvLBc' #川名のスプレッドシート
SP_SHEET = '周辺人口DB'
# スプレッドシート/シートを選択して初期化既存データ削除）
sh = gc.open_by_key(SP_SHEET_KEY)
worksheet = sh.worksheet(SP_SHEET)
worksheet.clear() 

# DataFrameをシートに書き込む
worksheet.update(values=[df.columns.values.tolist()] + df.values.tolist(), range_name='A1')

print('スプレッドシートへの書き込みが完了しました')




c:\Users\noriyasukawana\AppData\Local\Programs\Python\Python311\Lib\site-packages\gspread\worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


スプレッドシートへの書き込みが完了しました


In [4]:
# SQLでDBにデータ保存
# SQLiteデータベースへの接続
db_path = "STEP3チーム課題_TA_IndéMode_DB.db"
conn = sqlite3.connect(db_path) 

# データフレームをSQLiteデータベースに書き込む
table_name = '商圏人口構成'  # テーブル名を適切なものに変更してください
df.to_sql(table_name, conn, if_exists='replace', index=False)

print(db_path + ' への書き込みが完了しました')

STEP3チーム課題_TA_IndéMode_DB.db への書き込みが完了しました


>### 雑コード

In [2]:
import sqlite3

In [5]:
# SQLiteデータベースへの接続
db_path = "STEP3チーム課題_TA_IndéMode_DB.db"
conn = sqlite3.connect(db_path) 

# データフレームをSQLiteデータベースに書き込む
table_name = '商圏人口構成'  # テーブル名を適切なものに変更してください
df.to_sql(table_name, conn, if_exists='replace', index=False)

5

In [4]:
# sample_data.db がなかったら作成、あったら接続
# db_path = "STEP3チーム課題_TA_IndéMode_DB.db"
# conn = sqlite3.connect(db_path)

# テーブル作成関数
#def create_table():
#    sql = '''CREATE TABLE IF NOT EXISTS SAMPLE
#                 (ID    INTEGER PRIMARY KEY,
#                  TEST1 TEXT,
#                  TEST2 TEXT,
#                  TEST3 TEXT)'''

#    conn.execute(sql)
#    conn.commit()
#    conn.close()

# データ挿入関数
#def insert_record():
#    sql = "INSERT INTO SAMPLE VALUES (1, 'あ', 'い', 'う')"
#    conn.execute(sql)

#    sql = "INSERT INTO SAMPLE VALUES (2, 'てすと１', 'てすと２', 'てすと３')"
#    conn.execute(sql)

#    sql = "INSERT INTO SAMPLE VALUES (3, '12345', '22222', '55555')"
#    conn.execute(sql)

#    conn.commit()
#    conn.close()

# データ更新関数
#def update_record():
#    sql = "UPDATE SAMPLE SET TEST1 = 'か' WHERE ID = 1"
#    conn.execute(sql)
#    conn.commit()
#    conn.close()

# データ削除関数
#def delete_record():
#    sql = "DELETE FROM SAMPLE WHERE ID = 3"
#    conn.execute(sql)
#    conn.commit()
#    conn.close()

# データ検索関数
#def select_record():
#    sql = "SELECT * FROM SAMPLE WHERE ID = 2"
#    for row in conn.execute(sql):
#        print(row)

# 関数実行
#create_table()
#insert_record()

In [89]:
# ライブラリインポート
import requests
import urllib.request
import pandas as pd

In [4]:
# 国土地理院APIから緯度経度を取得
Address = "千葉市稲毛区轟町３丁目"
makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
s_quote = urllib.parse.quote(Address)
response = requests.get(makeUrl + s_quote)

#緯度を表示
print(response.json()[0]["geometry"]["coordinates"][0])
#経度を表示
print(response.json()[0]["geometry"]["coordinates"][1])

140.109146
35.62973


In [7]:
#jSTAT MAP認証設定
REQUEST_URL = 'https://jstatmap.e-stat.go.jp/statmap/api/1.00?category=richReport&func=getSummary'
USER_ID = '&userid=noriyasukawana@outlook.jp'  #個人の登録ID
API_KEY = '&key=dMUbbbyc9ThTzG4PNpA2'  #個人のAPIキー

# params入力設定
latitude = '&lat=' + str(response.json()[0]["geometry"]["coordinates"][1])  # 緯度
longitude = '&lng=' + str(response.json()[0]["geometry"]["coordinates"][0])  # 経度
rangeType = '&rangeType=driveTime'  # circle(円) or driveTime(到達圏)
travelMode = '&travelMode=walking'  # car(車) or walking(徒歩)
speed = '&speed=3.2'  # 時速(km/h)
time = '&time=15,30,45'  # 移動時間(min)
output = '&output=json'  # 出力形式
radius = '&radius=500'

In [13]:
# リクエスト
res = requests.get(REQUEST_URL + USER_ID + latitude + longitude + rangeType + travelMode + speed + time + API_KEY + output)
result = res.json()

In [14]:
result

{'GET_SUMMARY': {'RESULT': {'STATUS': 0,
   'ERROR_MSG': '正常終了しました。',
   'DATE': '2023-09-19 18:31:57'},
  'PARAMETER': {'LATITUDE': 35.62973,
   'LONGITUDE': 140.109146,
   'RANGE_TYPE': 'driveTime',
   'TRAVEL_MODE': 'walking',
   'SPEED': 3,
   'TIME': [{'@level': 3, '@unit': '分', '$': 40}]},
  'POSITION_INF': {'PREFECTURE': '千葉県', 'CITY': '千葉市稲毛区', 'BLOCK': '轟町三丁目'},
  'DATASET_INF': [{'TABLE_INF': [{'STATISTICS_NAME': '令和2年国勢調査',
      'STAT_KIND': '人口',
      'TITLE': 'TABLE01 男女別人口',
      'CLASS_INF': {'CLASS_OBJ': [{'@id': 'cat11',
         '@name': '集計範囲',
         'CLASS': [{'@code': 'R001', '@name': '1次エリア'},
          {'@code': 'R002', '@name': '2次エリア'},
          {'@code': 'R003', '@name': '3次エリア'},
          {'@code': 'R010', '@name': '千葉市稲毛区'},
          {'@code': 'R100', '@name': '千葉県'}]},
        {'@id': 'cat12',
         '@name': '男女',
         'CLASS': [{'@code': '1200', '@name': '男女計'},
          {'@code': '1201', '@name': '男'},
          {'@code': '1202', '@name':

In [119]:
#性別別人口
Gender_pop = result['GET_SUMMARY']['DATASET_INF'][0]['TABLE_INF'][0]['DATA_INF']['VALUE']
Gender_pop2 = [{key: value for key, value in entry.items() if key not in ['@cat11', '@cat12']} for entry in Gender_pop]
Gender_pop3 = [entry['$'] for entry in Gender_pop2]
# リストを3つずつに分割
list_Gender = [Gender_pop3[i:i+3] for i in range(0, len(Gender_pop3), 3)]
# DataFrameに変換
df_Gender = pd.DataFrame(list_Gender, columns=['総人口', '男性人口', '女性人口'])
display(df_Gender)

,総人口,男性人口,女性人口
0,3151,1588,1563
1,15776,8034,7742
2,73461,36662,36799
3,160582,80345,80237
4,6284480,3117987,3166493


In [120]:
#年齢別人口
Age_pop = result['GET_SUMMARY']['DATASET_INF'][0]['TABLE_INF'][1]['DATA_INF']['VALUE']
Age_pop2 = [{key: value for key, value in entry.items() if key not in ['@cat11', '@cat13']} for entry in Age_pop]
Age_pop3 = [entry['$'] for entry in Age_pop2]
# リストを3つずつに分割
list_Age = [Age_pop3[i:i+16] for i in range(0, len(Age_pop3), 16)]
# DataFrameに変換
df_Age = pd.DataFrame(list_Age, columns=['4 歳以下', '5～9 歳', '10～14 歳', '15～19 歳', '20～24 歳', '25～29 歳', '30～34 歳','35～39 歳', '40～44 歳','45～49 歳', '50～54 歳','55～59 歳', '60～64 歳','65～69 歳', '70～74 歳','75 歳以上'])
display(df_Age)

,4 歳以下,5～9 歳,10～14 歳,15～19 歳,20～24 歳,25～29 歳,30～34 歳,35～39 歳,40～44 歳,45～49 歳,50～54 歳,55～59 歳,60～64 歳,65～69 歳,70～74 歳,75 歳以上
0,109,138,159,192,252,158,127,161,181,278,206,213,181,143,157,269
1,460,571,636,831,1298,680,653,766,890,1102,961,942,814,890,966,1822
2,2112,2637,2910,3408,5173,3278,3186,3720,4420,5268,4714,4012,3589,3750,4350,8786
3,5273,6328,6977,7671,8592,7035,7416,8947,10536,12996,11391,9375,8330,9234,11050,21491
4,221053,248798,264645,281545,316219,311470,329858,372869,423399,507856,447662,382032,342781,387685,452539,859767


In [121]:
# DataFrameを横に結合
df = pd.concat([df_Gender, df_Age], axis=1)
display(df)

,総人口,男性人口,女性人口,4 歳以下,5～9 歳,10～14 歳,15～19 歳,20～24 歳,25～29 歳,30～34 歳,35～39 歳,40～44 歳,45～49 歳,50～54 歳,55～59 歳,60～64 歳,65～69 歳,70～74 歳,75 歳以上
0,3151,1588,1563,109,138,159,192,252,158,127,161,181,278,206,213,181,143,157,269
1,15776,8034,7742,460,571,636,831,1298,680,653,766,890,1102,961,942,814,890,966,1822
2,73461,36662,36799,2112,2637,2910,3408,5173,3278,3186,3720,4420,5268,4714,4012,3589,3750,4350,8786
3,160582,80345,80237,5273,6328,6977,7671,8592,7035,7416,8947,10536,12996,11391,9375,8330,9234,11050,21491
4,6284480,3117987,3166493,221053,248798,264645,281545,316219,311470,329858,372869,423399,507856,447662,382032,342781,387685,452539,859767


In [122]:
# 新しいカラムを作成
item_column = ['徒歩10分圏内', '徒歩20分圏内', '徒歩40分圏内', '区', '都道府県']

# 新しいカラムをDataFrameの先頭列に追加
df.insert(0, '項目', item_column)
display(df)

,項目,総人口,男性人口,女性人口,4 歳以下,5～9 歳,10～14 歳,15～19 歳,20～24 歳,25～29 歳,30～34 歳,35～39 歳,40～44 歳,45～49 歳,50～54 歳,55～59 歳,60～64 歳,65～69 歳,70～74 歳,75 歳以上
0,徒歩10分圏内,3151,1588,1563,109,138,159,192,252,158,127,161,181,278,206,213,181,143,157,269
1,徒歩20分圏内,15776,8034,7742,460,571,636,831,1298,680,653,766,890,1102,961,942,814,890,966,1822
2,徒歩40分圏内,73461,36662,36799,2112,2637,2910,3408,5173,3278,3186,3720,4420,5268,4714,4012,3589,3750,4350,8786
3,区,160582,80345,80237,5273,6328,6977,7671,8592,7035,7416,8947,10536,12996,11391,9375,8330,9234,11050,21491
4,都道府県,6284480,3117987,3166493,221053,248798,264645,281545,316219,311470,329858,372869,423399,507856,447662,382032,342781,387685,452539,859767


'千葉市稲毛区'

> まとめ